In [1]:
from __future__ import unicode_literals

import json
import os

# %pip install -U hazm
from hazm import *

os.chdir('../')

def read_file():
    documents_title = []
    documents_content = []
    documents_url = []
    with open(os.path.join(os.getcwd(), 'Phase_1', 'assets', 'IR_data_news_12k.json'), encoding='UTF-8') as f:
        data = json.load(f)
        for i in data:
            documents_title.append(data[i]["title"])
            documents_content.append(data[i]["content"])
            documents_url.append(data[i]['url'])
    return documents_url, documents_title, documents_content


docs_url, docs_title, docs_content = read_file()
normalizer = Normalizer()

In [2]:
# coding: utf8
from os import path
import codecs
from hazm.Normalizer import Normalizer
# from Phase_1.src.utils import data_path
default_stop_words = path.join(os.getcwd(), 'Phase_1', 'src', 'data', 'stopwords.dat')


class StopWord:
    """ Class for remove stop words

         >>> StopWord().clean(["در","تهران","کی","بودی؟"])
         ['بودی؟', 'تهران', 'کی']
         >>> StopWord(normal=True).clean(["در","تهران","کی","بودی؟"])
         ['بودی؟', 'تهران']

         """

    def __init__(self, file_path=default_stop_words, normal=False):
        self.file_path = file_path
        self.normal = normal
        self.normalizer = Normalizer().normalize
        self.stop_words = self.init(file_path, normal)

    def init(self, file_path, normal):
        if not normal:
            return set(
                line.strip("\r\n") for line in codecs.open(file_path, "r", encoding="utf-8").readlines())
        else:
            return set(
                self.normalizer(line.strip("\r\n")) for line in
                codecs.open(file_path, "r", encoding="utf-8").readlines())

    def set_normalizer(self, func):
        self.normalizer = func
        self.stop_words = self.init(self.file_path, self.normal)

    def __getitem__(self, item):
        return item in self.stop_words

    def __str__(self):
        return str(self.stop_words)

    def clean(self, iterable_of_strings, return_generator=False):
        if return_generator:
            return filter(lambda item: not self[item], iterable_of_strings)
        else:
            return list(filter(lambda item: not self[item], iterable_of_strings))


In [3]:
def preprocess_content(content):
    str_empty = ' '
    stemmer = Stemmer()
    content = normalizer.normalize(content)
    content = word_tokenize(content)
    content = StopWord(normal=False).clean(content)
    content = [stemmer.stem(word) for word in content]
    content = str_empty.join(content)
    return content

In [86]:
for i in range(len(docs_content)):
    docs_content[i] = preprocess_content(docs_content[i])

In [87]:
from datetime import datetime
from elasticsearch import Elasticsearch, helpers

# Connect to 'http://localhost:9200'
es = Elasticsearch("http://localhost:55001")

In [90]:
import uuid

def generate_data():
    data = []
    for doc_content, doc_url, doc_title in zip(docs_content, docs_url, docs_title):
        if doc_content and doc_url and doc_title:        
            data.append({
                "_index": "ir-news-index",
                "_id" : uuid.uuid4(),
                "_source": {
                    "content": doc_content,
                    "url": doc_url,
                    "title": doc_title
                }
            })
    return data

data=generate_data()
response = helpers.bulk(es, data)

گزار خبرگزار فارس کنفدراسیون فوتبال آسیا AFC نامه رسم فدراسیون فوتبال ایر باشگاه گیت پسند قرعه کش جا باشگاه فوتسال آسیا اساس ۲۵ فروردین ماه ۱۴۰۱ مراس قرعه کش جا باشگاه فوتسال آسیا مالز برگزار باشگاه گیت پسند بعنو قهر فوتسال ایر سال ۱۴۰۰ مسابق کرده_اس گیت پسند تجربه ۳ دوره حضور جا باشگاه فوتسال آسیا سه دوره فینال مسابق قهرمان مقا دوم بدس آورده_اس انت پیام/


In [151]:
def calc_string(string):
    print(string)
    state = False
    i = 0
    while i < len(string):
        ch = string[i]
        if ch == "\"":
            if state:
                string = string[:i] + '<$>' + string[i + 1:]
            else:
                string = string[:i] + '<#>' + string[i + 1:]
            state = 1 ^ state
        i += 1
    string = str.replace(string, ' ! ', 'NOT')
    string = str.replace(string, ' + ', 'AND')
    string = str.replace(string, ' ', 'OR')
    state = False
    i = 0
    while i < len(string):
        ch = string[i]
        if ch == "(":
            state = True
        elif ch == ')':
            state = False
        if state and ch == 'O':
            string = string[:i] + ' ' + string[i + 2:]
        i += 1
    string = str.replace(string, 'NOT', ' NOT ')
    string = str.replace(string, 'AND', ' AND ')
    string = str.replace(string, 'OR', ' OR ')
    return string

def preprocess_pipeline(query):
    str_empty = ' '
    stemmer = Stemmer()
    query = normalizer.normalize(query)
    query = word_tokenize(query)
    query = StopWord(normal=False).clean(query)
    query = [stemmer.stem(word) for word in query]
    query = str_empty.join(query)
    return query


def preprocess_query(query):
    query = calc_string(query)
    preprocessed_query = preprocess_pipeline(query)
    preprocessed_query = str.replace(preprocessed_query, '<$>', ')')
    preprocessed_query = str.replace(preprocessed_query, '<#>', '(')
    preprocessed_query = str.replace(preprocessed_query, 'OR OR', 'OR')
    preprocessed_query = str.replace(preprocessed_query, 'OR AND', 'AND')
    preprocessed_query = str.replace(preprocessed_query, 'OR NOT', 'NOT')
    preprocessed_query = str.replace(preprocessed_query, 'AND OR', 'AND')
    preprocessed_query = str.replace(preprocessed_query, 'AND AND', 'AND')
    preprocessed_query = str.replace(preprocessed_query, 'NOT AND', 'AND')
    preprocessed_query = str.replace(preprocessed_query, 'NOT OR', 'OR')
    preprocessed_query = str.replace(preprocessed_query, 'NOT NOT', 'NOT')
    print("query: ", preprocessed_query)
    
    return preprocessed_query

def query_data(query):
    query_to_ask_es = preprocess_query(query)
    res = es.search(index="ir-news-index", body={"query": {"query_string": { "default_field": "content",
                                                                             "query": query_to_ask_es
                                                                        }}})
    return res

In [152]:
query_data('تحریم‌های آمریکا علیه "جمهوری اسلامی ایران" ! آلمان + انگلیس').get('hits')

تحریم‌های آمریکا علیه "جمهوری اسلامی ایران" ! آلمان + انگلیس
query:  تحریم OR آمریکا OR (جمهور OR اسلام OR ایران) NOT آل AND انگلیس


/var/folders/jt/0d5khnwn23n745h24mhzcvxh0000gn/T/ipykernel_44290/3128248231.py:63: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index="ir-news-index", body={"query": {"query_string": { "default_field": "content",


{'total': {'value': 550, 'relation': 'eq'},
 'max_score': 20.232433,
 'hits': [{'_index': 'ir-news-index',
   '_id': '95062374-6077-471c-96ca-24157ed7a4e1',
   '_score': 20.232433,
   '_ignored': ['content.keyword'],
   '_source': {'content': 'روبر بیگلر نماینده مسیح ارمن جنوب ایر مجلس شورا اسلام گفت\u200cو خبرنگار خبرگزار فارس اشاره دادگاه ساختگ انگلیس موضوع حوادث آب ماه ۹۸ انگلیس سو همکار آمریکا تحریم اقتصاد ایر اعمال می\u200cکند تشکیل دادگاه دنبال اعمال فشار رسانه جمهور اسلام ایر افزود هدف انگلیس تشکیل دادگاه ساختگ پرونده\u200cساز جمهور اسلام ایر خیال جامعه ایر خاطر تحریم اقتصاد دچار آشوب نیرو انتظام پلیس ایر خاطر فضاسازی سنگین نتوانند برخورد آشوبگر داشته_باشند نماینده مسیح ارمن جنوب ایر مجلس اظهار  علاوه انگلیس اقدا فشار روان ایر مذاکر قالب برجا تشدید فضا روان سنگین دیپلمات کشور پا میز مذاکره انگلیس بیگلر خاطرن انگلیس نق پررنگ حوادث آب ماه سال ۹۸ موجب آشوب خیابان کشور فراه اقدا رد پا آشوب پاک انگلیس می\u200cخواهد اقدا اتهامات تبرئه سازد پاسخگو فضا روان سنگین ایر اندازد نماینده مسیح

In [153]:
query_data('"تحریم هستهای" آمریکا ! ایران').get('hits')

"تحریم هستهای" آمریکا ! ایران
query:  (تحر OR هستهای) OR آمریکا NOT ایر


/var/folders/jt/0d5khnwn23n745h24mhzcvxh0000gn/T/ipykernel_44290/3128248231.py:63: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index="ir-news-index", body={"query": {"query_string": { "default_field": "content",


{'total': {'value': 276, 'relation': 'eq'},
 'max_score': 9.315659,
 'hits': [{'_index': 'ir-news-index',
   '_id': 'b82eff34-3e60-43e9-9f5f-e4afebfe8fdb',
   '_score': 9.315659,
   '_ignored': ['content.keyword'],
   '_source': {'content': 'غلامرضا منتظر نماینده گرگ مجلس شورا اسلام گفت\u200cوگو خبرنگار پارلمان خبرگزار فارس اشاره مذاکر لغو تحر لزو هوشیار ت مذاکره\u200cکننده ایران تجربه آمریکا کشور بدعهد نمی\u200cتو اعتماد ادامه بحث مذاکر نماینده آمریکا تضمین لغو تحریم نمی\u200cداد ضمانت دول بعد آمریکا توافق پایبند نماینده گرگ مجلس شورا اسلام اظهار رهبر معظ انقلاب سه اصل عز مصلح حکم را\u200c\u200c\u200c\u200c\u200c\u200c\u200c اصول سیاس خارج نظر مذاکره عز حفظ منتظر خاطرن مسائل معیشت اقتصاد کشور رهبر معظ انقلاب آقا رئیس جمهور تصریح بحث مذاکر گره بخورد اجازه اقتصاد کشور شرط افزود تکیه تو ظرفیت داخل نیاز اقتصاد را\u200c\u200c\u200c\u200c\u200c\u200c\u200c برطرف ساز ظرف کشور انت پیام/\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c

In [154]:
query_data('اورشلیم ! صهیونیست').get('hits')

اورشلیم ! صهیونیست
query:  اورشل NOT صهیونیس


/var/folders/jt/0d5khnwn23n745h24mhzcvxh0000gn/T/ipykernel_44290/3128248231.py:63: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index="ir-news-index", body={"query": {"query_string": { "default_field": "content",


{'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}